# Exploring Wikipedia as a dataset for tar/inc tagging model
This notebook walks through the methods used to build a dataste of wikipedia pages to train our relational tagging model.

In [2]:
# standard library imports
import sys
import re
sys.path.append('../')
# third party imports
import wikipedia
from tqdm import tqdm
from shapely.geometry import Point, shape
from geopy.distance import distance
import pandas as pd
# local imports
from BERT_geoparser.gazetteer import Gazetteer
from BERT_geoparser.tokenizer import Tokenizer
from BERT_geoparser.model import BertModel
from BERT_geoparser.data import Data

## 2 Building a dataset of wikipedia locations
Now we have a trained NER-tagging model we can move on to buildng a new dataset to tage. To do this, we will randomly sample locations from a gazetteer of world cities, and use the wikipedia api to access the corresponding wikipedia page. We will then randomly sample sentences from that page an split them into tokens to input into the model.

# 2.1 Building a list of locations to search for
We have used work by Hertford et al (Hertford, 2023) to limit our search to countries which have relatively high OSM completeness. These countries are stored in the parameter `accepted_countries`. We then use the GeoNames datasbase to search for all cities with a population greater than 1000 within those countries. This dataset requires a little cleaning before use.

In [71]:
world_cities = pd.read_csv('../data/NB2/geonames_world_cities.csv', delimiter=';')
world_cities.tail()

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Country Code 2,Admin1 Code,Admin2 Code,Admin3 Code,Admin4 Code,Population,Elevation,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates
142027,1694792,Paraiso,Paraiso,Paraiso,P,PPL,PH,Philippines,NaN,12,70,126306000,NaN,4153,NaN,90,Asia/Manila,2018-02-06,Philippines,"6.47606, 124.80954"
142028,1694812,Paracale,Paracale,Paracale,P,PPLA3,PH,Philippines,NaN,05,15,051608000,NaN,8290,NaN,5,Asia/Manila,2017-12-13,Philippines,"14.2804, 122.7881"
142029,1695427,Pangal Sur,Pangal Sur,"Pangal,Pangal Sur",P,PPL,PH,Philippines,NaN,02,31,023112000,NaN,2710,NaN,93,Asia/Manila,2017-12-13,Philippines,"16.60036, 121.66979"
142030,1695804,Panabo,Panabo,"Kota Panabo,Panabo,Panabo City,Panabo Lakanbalen",P,PPL,PH,Philippines,NaN,11,I7,112315000,NaN,84749,NaN,4,Asia/Manila,2017-12-13,Philippines,"7.30806, 125.68417"
142031,1695994,Palsong,Palsong,Palsong,P,PPL,PH,Philippines,NaN,05,16,051706000,NaN,4566,NaN,18,Asia/Manila,2017-12-13,Philippines,"13.4248, 123.2963"


In [9]:
# clean up the coordinates column
get_lat = lambda coords : float(coords.split(', ')[0])
get_lng = lambda coords : float(coords.split(', ')[1])

world_cities['lat'] = world_cities.Coordinates.apply(get_lat)
world_cities['lng'] = world_cities.Coordinates.apply(get_lng)
world_cities = world_cities.rename(columns={'Name':'city', 'ASCII Name':'city_ascii', 'Country name EN':'country', 'Population':'population'})

In [10]:
countries = world_cities.country.unique()

In [11]:
accepted_countries = ['United States', 'Canada', 'Portugal', 'Spain', 'France', 'Ireland', 'United Kingdom', 'Norway', 'Sweden', 'Denmark', 'Finland',
                      'Belgium', 'Netherlands', 'Switzerland', 'Austria', 'Germany', 'Italy', 'Czech Republic', 'Slovakia', 'Slovenia', 'Hungary', 'Bosnia and Herzegovina', 'Croatia',
                      'Slovakia', 'Poland', 'Lithuania', 'Latvia', 'Estonia', 'Iceland']

In [12]:
# check all the countries are in the world_cities list
for country in accepted_countries:
    if country not in world_cities.country.unique():
        print(country)

In [14]:
# cut the list of cities down to just those in our list of accepted countries
world_cities = world_cities[world_cities.country.isin(accepted_countries)]
print(f'Total locations in dataset: {len(world_cities)}')

Total locations in dataset: 72757


## 2.1 Checking that locations have associated OSM polygons
Our method for relational tagging is reliant on the target locations being associated with a spatial polygon in OSM.. We will use the `Gazetteer` class to search the Nominatim API for each locaiton, and check whether a polygon object is returned. This can take some time, depnding on the speed of your internet connect. It took about 90 mins to complete for me. 

In [208]:
# We're going to use the gazetteer to limit these to cities with OSM polygons
gazetteer = Gazetteer(polygon=True, addressdetails=False)
accepted_indices = []
for i, city in tqdm(world_cities.iterrows(), total=len(world_cities)):
    name = city.city
    coords = (float(city.lat), float(city.lng))
    country = city.country
    try:
        matches = gazetteer.query(name).json()
    except TypeError:
        continue
    for match in matches:
        if 'geojson' not in match.keys():
            continue
        if 'Polygon' not in match['geojson']['type']:
            continue
        if Point(coords) in shape(match['geojson']):
            accepted_indices.append(i)
            break
# cut or cdity list down to just the ones with polygons, remove duplicates.
poly_cities = world_cities.loc[accepted_indices]
deduplicated = poly_cities.drop_duplicates()
print(len(deduplicated))   
print(poly_cities.head())         

100%|██████████| 10851/10851 [1:30:44<00:00,  1.99it/s]


In [216]:
# save the dataset
deduplicated.to_csv('../data/NB2/world_cities.csv', index=False)

## 2.3 Searching wikipedia for these locations
We now need to search the Wikipeida API for locations matching these. We will do various check to ensure that the identified page matches the expected location, including checking for coordinates, checking for key words, and checking for disambiguation pages.

First, though, let's take a look at the data provided by the Wikipedia wpi.

In [15]:
# lets take a quick look at the wikipedia api
glasgow = wikipedia.WikipediaPage(title='Glasgow')

In [16]:
glasgow.summary

'Glasgow (UK:  GLA(H)Z-goh, GLA(H)SS-; Scots: Glesca [ˈɡleskə] or Glesga [ˈɡlezɡə]; Scottish Gaelic: Glaschu [ˈkl̪ˠas̪əxu]) is the most populous city in Scotland, and is the third-most populous city in the United Kingdom, as well as being the 27th largest city by population in Europe. In 2022, it had an estimated population as a defined locality of \n632,350 and anchored an urban settlement of \n1,028,220. Formed as a county of itself in 1893, the city had previously been in the historic county of Lanarkshire (or Clydesdale) and has also grown to include settlements that were once part of Renfrewshire and Dunbartonshire (or the Lennox). It now forms the Glasgow City Council area, one of the 32 council areas of Scotland, and is administered by Glasgow City Council. \nGlasgow has the largest economy in Scotland and the third-highest GDP per capita of any city in the UK. Glasgow\'s major cultural institutions enjoy international reputations including The Royal Conservatoire of Scotland, B

In [17]:
# we'll need a function to remove anything inside brackets to avoid the pronounciation gunk
def remove_parentheses(input_string):
    result = ""
    stack = []
    for char in input_string:
        if char in ['(', '[']:
            stack.append(char)
        elif char in [')', ']']:
            if stack:
                stack.pop()
            else:
                result += char
        elif not stack:
            result += char
    
    return result.replace('  ', ' ')

In [18]:
out = remove_parentheses(glasgow.summary)
print(out)

Glasgow is the most populous city in Scotland, and is the third-most populous city in the United Kingdom, as well as being the 27th largest city by population in Europe. In 2022, it had an estimated population as a defined locality of 
632,350 and anchored an urban settlement of 
1,028,220. Formed as a county of itself in 1893, the city had previously been in the historic county of Lanarkshire and has also grown to include settlements that were once part of Renfrewshire and Dunbartonshire . It now forms the Glasgow City Council area, one of the 32 council areas of Scotland, and is administered by Glasgow City Council. 
Glasgow has the largest economy in Scotland and the third-highest GDP per capita of any city in the UK. Glasgow's major cultural institutions enjoy international reputations including The Royal Conservatoire of Scotland, Burrell Collection, Kelvingrove Art Gallery and Museum, Royal Scottish National Orchestra, BBC Scottish Symphony Orchestra, Scottish Ballet and Scottish

In [19]:
# and lets look at where disambiguation pages land
hull = wikipedia.page('Hull')
hull.categories

c:\Users\jws10y\AppData\Local\anaconda3\envs\tensorflow-gpu\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\jws10y\AppData\Local\anaconda3\envs\tensorflow-gpu\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "tall" may refer to: 
height
human height
Tall, Semnan
River Tall
Tall: The American Skyscraper and Louis Sullivan
Mr. Tall
Tall (surname)
Tell (archaeology)
List of people known as the Tall
TAL (disambiguation)
Tell (disambiguation)
All pages with titles beginning with tall
All pages with titles beginning with tallest
All pages with titles containing tall

In [20]:
# all disambiguation pages lead to a page error, lets catch that 
def disamb_check(query):
    try:
        wikipedia.WikipediaPage(query)
        return False
    except (wikipedia.DisambiguationError, wikipedia.PageError):
        return True


In [21]:
check = disamb_check('hull')
check

True

In [22]:
check = disamb_check('glasgow')
check

False

We will also need a way to check that the found page refers to a city and not to something else. Unfortunately the wikipedia 'Location' category is not applied consistently enough to be suitable for this. Rather, we'll use a slightly hacky approach in which either use the coordinates provided on the iwkipedis page (if available), or other look for words related to locations within the first two sentences of the page.


In [23]:
# first we should check if the coordinates are roughly correct
def check_coordinates(page, true_coords, threshold=10):
    if not page.coordinates:
        return False
    wiki_coords = page.coordinates
    d = distance(true_coords, wiki_coords).km
    return d <= threshold
    
    

In [26]:
# Lets test it with the coordinates for Birmingham Alabama
page = wikipedia.WikipediaPage('Birmingham')
true_coords = (33.52066, -86.80249)

check_coordinates(page, true_coords)

False

In [27]:
page = wikipedia.WikipediaPage('Birmingham, AL')
true_coords = (33.52066, -86.80249)

check_coordinates(page, true_coords)

True

In [43]:
# looking for words related to cities in the first sentence (usually X is a city in Y)
city_words = ['city', 'town', 'village', 'township', 'commune', 'community', 'settlement', 'district', 'municipality', 'metropolis', 'conurbation']

def check_is_city(page):
    sentences = page.summary.split('.')
    out = False
    for word in city_words:
        if word in sentences[0].lower():
            out = True
    return out

In [47]:
# Looking for mentions of the country the city is situated in
def check_correct_country(page, country):
    sentences = page.summary.split('.')
    for sentence in sentences[:3]:
        if country in sentence:
            return True
    return False

In [48]:
# lets test this by looking for Paradise, California
page = wikipedia.WikipediaPage('paradise')

check_is_city(page)

False

In [49]:
page = wikipedia.WikipediaPage('paradise (California)')

check_is_city(page)

True

In [55]:
# And we need a check to see if the page exists at all.
def check_page_exists(query):
    try:
        wikipedia.WikipediaPage(query)
        return True
    except wikipedia.PageError:
        return False

Some of the wikipedia summaries are formatted so that line breaks can appear as `<word>.<word>` in the summary. We need to put in a space after the period when this occurs

In [56]:
def add_space_after_period(text):
    pattern = r'([A-Za-z])\.([A-Za-z])'
    replacement = r'\1. \2'
    result = re.sub(pattern, replacement, text)
    return result

finally, we'll need a function which can split a piece of text into full sentences, up to the `max_len` value in the model.

In [57]:
import nltk
nltk.download('punkt')

def split_into_sentences(text, max_words=60):
    sentences = nltk.sent_tokenize(text)
    result = []
    current_sentence = ""

    for sentence in sentences:
        if len(current_sentence.split()) + len(sentence.split()) <= max_words:
            current_sentence += " " + sentence
        else:
            result.append(current_sentence.strip())
            current_sentence = sentence

    # Add the last substring
    if current_sentence:
        result.append(current_sentence.strip())

    return result

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jws10y\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
sentences = split_into_sentences(glasgow.summary)
sentences

['Glasgow (UK:  GLA(H)Z-goh, GLA(H)SS-; Scots: Glesca [ˈɡleskə] or Glesga [ˈɡlezɡə]; Scottish Gaelic: Glaschu [ˈkl̪ˠas̪əxu]) is the most populous city in Scotland, and is the third-most populous city in the United Kingdom, as well as being the 27th largest city by population in Europe.',
 'In 2022, it had an estimated population as a defined locality of \n632,350 and anchored an urban settlement of \n1,028,220. Formed as a county of itself in 1893, the city had previously been in the historic county of Lanarkshire (or Clydesdale) and has also grown to include settlements that were once part of Renfrewshire and Dunbartonshire (or the Lennox).',
 'It now forms the Glasgow City Council area, one of the 32 council areas of Scotland, and is administered by Glasgow City Council. Glasgow has the largest economy in Scotland and the third-highest GDP per capita of any city in the UK.',
 "Glasgow's major cultural institutions enjoy international reputations including The Royal Conservatoire of S

In [59]:
# This function draws together each of the page processing functions. 
def process_page(page):
    raw_summary = page.summary
    p1_summary = remove_parentheses(raw_summary)
    p2_summary = add_space_after_period(p1_summary)
    sentences = split_into_sentences(p2_summary, 70)
    return sentences

 We now have all the funcitons we need to scrape Wikpedia for the locations in the dataset. This takes a long time and can run into problems if an internet connection drops out. It took around 5 hours to complete for me.

In [ ]:
# we can now loop over each of our cities and find the wikipedia page associated with each
rows = []
for idx, row in tqdm(poly_cities.iterrows(), total = len(poly_cities)):
    location = row.city_ascii
    state = row['Admin1 Code']
    country = row.country
    coords = (row.lat, row.lng)
    
    # check for disambiguation and add state/country if disamb reached
    if disamb_check(location):
        # pages for US locations work better if you add the state
        if country == 'United States':
            location += f' ({state})'
        else:
            location += f' ({country})'
        # if the new location doesn't have a page then skip
        if disamb_check(location):
            continue
    
    ## check a page exists related to that location
    if not check_page_exists(location):
        continue
    
    # retrieve page
    page = wikipedia.WikipediaPage(location)
    # check if the page has coordinates:
    try:
        page.coordinates
        has_coords = True
    except KeyError:
        has_coords = False
        correct_coords = False
    # check the coordinates match
    if (has_coords) and not (check_coordinates(page, coords)):
        if country == 'united_states':
            location += state
        else:
            location += f'({country})'
        # check a page exists, skip otherwise
        if check_page_exists(location):  
            page = wikipedia.page(location)
            correct_coords = False
            # check again with the new page, if still a miss then skip
            if not check_coordinates(page, coords):
                continue
        else:
            continue

    elif (has_coords) and (check_coordinates(page, coords)):
        correct_coords = True
        
    # check country is correct
    if not (correct_coords) and not (check_correct_country(page, country)):
        location += f'({country})'
        # check a page exists, skip otherwise
        if check_page_exists(location):  
            page = wikipedia.page(location)
        else:
            continue
    
    # check this is a page for a city
    if not (correct_coords) and not (check_is_city(page)):
        continue 
    
    # If it gets this far then the wikipedia page is confirmed as the correct one! 
    sentences = process_page(page)
    for sentence in sentences:
        row = {}
        row['city'] = location
        row['country'] = country
        row['coordinates'] = coords
        row['text'] = sentence
        rows.append(row)
        
processed_wiki_cities = pd.DataFrame(rows)
        
processed_wiki_cities.to_csv('../data/NB2/wiki_cities_reference.csv', index=False)    

## 2.3 Adding US Counties
The current dataset is made mostly of towns and cities. As such, there is likely to be limited representation of [CHI] tags in the data. We will address this by adding US counties dataset. Once agian, we need to process these to ensure they can be associated to an OSM polygon and a valid Wikipedia page. This takes a couple of hours to run.

In [72]:
us_counties = pd.read_csv('../data/NB2/us_counties.csv', encoding='latin')
us_counties.head()

clean_latitude = lambda lat : float(lat[1:-1])
clean_longitude = lambda lng : -float(lng[1:-1])

us_counties['Latitude'] = us_counties['Latitude'].apply(clean_latitude)
us_counties['Longitude'] = us_counties['Longitude'].apply(clean_longitude)
us_counties = us_counties.rename(columns={'County [2]':'County'})

In [73]:
us_counties

,Sort [1],State,FIPS,County,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,1,AL,1001,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,32.536382,-86.644490
1,2,AL,1003,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",30.659218,-87.746067
2,3,AL,1005,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,31.870670,-85.405456
3,4,AL,1007,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,33.015893,-87.127148
4,5,AL,1009,Blount,Oneonta,"57,322","1,669.96",644.776,15.157,5.852,"1,685.12",650.628,33.977448,-86.567246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3139,WY,56037,Sweetwater,Green River,"43,806","27,004.90","10,426.65",166.887,64.436,"27,171.78","10,491.09",41.660339,-108.875676
3139,3140,WY,56039,Teton,Jackson,"21,294","10,347.98","3,995.38",572.266,220.953,"10,920.25","4,216.33",44.049321,-110.588102
3140,3141,WY,56041,Uinta,Evanston,"21,118","5,390.45","2,081.26",16.342,6.31,"5,406.79","2,087.57",41.284726,-110.558947
3141,3142,WY,56043,Washakie,Worland,"8,533","5,797.82","2,238.55",10.762,4.155,"5,808.58","2,242.70",43.878831,-107.669052


In [74]:
from shapely.geometry import shape
# We're again going to use the gazetteer to limit these to counties with OSM polygons
gazetteer = Gazetteer(polygon=True, addressdetails=False)
accepted_indices = []
for i, county in tqdm(us_counties.iterrows(), total=len(us_counties)):
    name = county.County
    coords = (float(county.Longitude), float(county.Latitude))
    state = county.State
    try:
        matches = gazetteer.query(name).json()
    except TypeError:
        continue
    for match in matches:
        if 'geojson' not in match.keys():
            continue
        if 'Polygon' not in match['geojson']['type']:
            continue
        if Point(coords).within(shape(match['geojson'])):
            accepted_indices.append(i)
            break

  0%|          | 0/3143 [00:00<?, ?it/s]

100%|██████████| 3143/3143 [27:43<00:00,  1.89it/s] 


In [75]:
location = 'Autauga County'
if disamb_check(location):
    new_location = location + f', {state}'
    
    # if the new location doesn't have a page then skip
    if disamb_check(new_location):
        print(f'"{location}" leads to disambiguation')

In [76]:
wikipedia.WikipediaPage('Autauga County, AL')

<WikipediaPage 'Autauga County, Alabama'>

In [77]:
poly_counties = us_counties.loc[list(set(accepted_indices))]

In [78]:
rows = []
for idx, row in tqdm(poly_counties.iterrows(), total = len(poly_counties)):
    location = remove_parentheses(row.County + ' County')
    state = row['State']
    country = 'United States'
    coords = (row.Latitude, row.Longitude)
    
    location += f', {state}'
    # check for disambiguation and add state/country if disamb reached
    if disamb_check(location):
        print(f'Page for "{location}" cannot be disambiguated and/or found.')
        continue
    # retrieve page
    page = wikipedia.WikipediaPage(location)   
    sentences = process_page(page)
    for sentence in sentences:
        row = {}
        row['city'] = location
        row['country'] = country
        row['coordinates'] = coords
        row['text'] = sentence
        rows.append(row)
        
processed_wiki_counties = pd.DataFrame(rows)

  2%|▏         | 62/2817 [02:20<1:20:22,  1.75s/it]

Page for "Aleutians East County, AK" cannot be disambiguated and/or found.


  2%|▏         | 63/2817 [02:21<1:02:10,  1.35s/it]

Page for "Anchorage County, AK" cannot be disambiguated and/or found.


  2%|▏         | 64/2817 [02:21<49:44,  1.08s/it]  

...

 94%|█████████▍| 2657/2817 [1:40:52<01:46,  1.50it/s]

Page for "Poquoson County, VA" cannot be disambiguated and/or found.


 94%|█████████▍| 2658/2817 [1:40:52<01:37,  1.62it/s]

Page for "Virginia Beach County, VA" cannot be disambiguated and/or found.


100%|██████████| 2817/2817 [1:47:05<00:00,  2.28s/it]


In [79]:
processed_wiki_counties.to_csv('../data/NB2/wiki_counties_reference.csv', index = False)

## 2.4 Tagging locations in the datasets
Finally, we need to use our trained TopoBERT model to identify toponyms in each dataset. The two datasets will be joined together after relational retagging in notebook 3.


In [ ]:
# Load the dataset using the BERT_geoparser Data.py module
data_csv = r'../data/NB1/wikineural_train_dataset.csv' # specify the data used to train the model for correct tokenization.
tokenizer = Tokenizer(size='large', cased=True)
data = Data(data_path=data_csv, 
            tokenizer=tokenizer,
            max_len=80)

model = BertModel(saved_model='../models/TopoBERT_WikiNeural.hdf5', data=data)
model.model.summary()

cities_data = pd.read_csv('../data/NB2/wiki_cities_reference.csv')
counties_data = pd.read_csv('../data/NB2/wiki_counties_reference.csv')

print('Tagging cities dataset')
cities_results_df = model.results_dataframe(cities_data.text.values, include_best=True)
print('Tagging counties dataset')
counties_results_df = model.results_dataframe(counties_data.text.values, include_best=True)

In [48]:
cities_results_df.to_csv('../data/NB2/wiki_cities_tagged.csv', index=False)
counties_results_df.to_csv('../data/NB2/wiki_counties_tagged.csv', index=False)